In [2]:
import cv2
import os
import numpy as np
from PIL import Image
from keras.models import load_model
from keras.utils import normalize
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score

# Load the pre-trained model
model = load_model('BrainTumor10EpochsCategorical.h5')

# Set up the directory for the new dataset
new_image_directory = 'brain_tumor_dataset/'

# Load images
new_no_tumor_images = os.listdir(new_image_directory + 'no')
new_yes_tumor_images = os.listdir(new_image_directory + 'yes')
new_dataset = []
new_label = []

INPUT_SIZE = 64

# Preprocess no tumor images
for image_name in new_no_tumor_images:
    if image_name.endswith('.jpg') or image_name.endswith('.jpeg') or image_name.endswith('.png'):
        # Assuming filename format is [i]
        image = cv2.imread(new_image_directory + 'no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        new_dataset.append(np.array(image))
        new_label.append(0)

# Preprocess yes tumor images
for image_name in new_yes_tumor_images:
    if image_name.endswith('.jpg') or image_name.endswith('.jpeg') or image_name.endswith('.png'):
        # Assuming filename format is Y[i]
        image = cv2.imread(new_image_directory + 'yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        new_dataset.append(np.array(image))
        new_label.append(1)

new_dataset = np.array(new_dataset)
new_label = np.array(new_label)

# Normalize the dataset
new_dataset = normalize(new_dataset, axis=1)

# Convert labels to categorical
new_label = to_categorical(new_label, num_classes=2)

# Predictions
new_pred_prob = model.predict(new_dataset)
new_pred = np.argmax(new_pred_prob, axis=1)
new_true = np.argmax(new_label, axis=1)

# Calculate accuracy
accuracy = accuracy_score(new_true, new_pred) * 100
print(f'Accuracy: {accuracy:.2f}%')

# Generate classification report
report = classification_report(new_true, new_pred)
print("Classification Report:")
print(report)

# Confusion Matrix
cm = confusion_matrix(new_true, new_pred)
print("Confusion Matrix:")
print(cm)

# Calculate ROC-AUC
roc_auc = roc_auc_score(new_label[:, 1], new_pred_prob[:, 1])
print(f'ROC-AUC Score: {roc_auc:.2f}')

# Calculate F1 Score
f1 = f1_score(new_true, new_pred)
print(f'F1 Score: {f1:.2f}')


c:\Users\truon\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Accuracy: 97.21%
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97        92
           1       0.97      0.98      0.97        87

    accuracy                           0.97       179
   macro avg       0.97      0.97      0.97       179
weighted avg       0.97      0.97      0.97       179

Confusion Matrix:
[[89  3]
 [ 2 85]]
ROC-AUC Score: 0.99
F1 Score: 0.97


In [1]:
import cv2
import os
import numpy as np
from PIL import Image
from keras.models import load_model
from keras.utils import normalize
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             f1_score, accuracy_score, precision_score, recall_score)

# Load the pre-trained model
model = load_model('BrainTumor10EpochsCategorical.h5')

# Set up the directory for the test dataset
test_image_directory = 'pred'

# Load images from the test dataset
test_images = os.listdir(test_image_directory)
test_dataset = []

INPUT_SIZE = 64

# Preprocess test images
for image_name in test_images:
    if image_name.endswith('.jpg') or image_name.endswith('.jpeg') or image_name.endswith('.png'):
        image_path = os.path.join(test_image_directory, image_name)  # Get the full path
        image = cv2.imread(image_path)

        # Check if the image was loaded successfully
        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip this iteration

        # Convert to RGB and resize
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        test_dataset.append(np.array(image))

test_dataset = np.array(test_dataset)

# Check if the dataset is empty after loading images
if len(test_dataset) == 0:
    print("No valid images found in the test dataset.")
else:
    # Normalize the test dataset
    test_dataset = normalize(test_dataset, axis=1)

    # Make predictions
    pred_prob = model.predict(test_dataset)

    # Set a threshold for classification
    threshold = 0.5
    predictions = [1 if prob[1] > threshold else 0 for prob in pred_prob]  # 1 for "yes", 0 for "no"

    # Simulating true labels for the sake of evaluation
    # You should replace this with actual labels when available
    true_labels = np.random.randint(2, size=len(predictions))  # Random labels for demonstration

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, pred_prob[:, 1])  # Use probabilities for AUC
    cm = confusion_matrix(true_labels, predictions)

    # Count total images, correct predictions, and incorrect predictions
    total_images = len(test_images)
    correct_predictions = np.sum(predictions == true_labels)
    incorrect_predictions = total_images - correct_predictions
    accuracy_percentage = (correct_predictions / total_images) * 100 if total_images > 0 else 0

    # Output the results
    print(f'Total Images: {total_images}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Incorrect Predictions: {incorrect_predictions}')
    print(f'Accuracy Percentage: {accuracy_percentage:.2f}%')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')
    print(f'ROC-AUC Score: {roc_auc:.2f}')
    print("Confusion Matrix:")
    print(cm)

    # Output predictions
    for i, image_name in enumerate(test_images):
        print(f'Image: {image_name}, Prediction: {"yes" if predictions[i] else "no"}, Probability: {pred_prob[i]}')


c:\Users\truon\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Total Images: 60
Correct Predictions: 30
Incorrect Predictions: 30
Accuracy Percentage: 50.00%
Accuracy: 0.50
Precision: 0.62
Recall: 0.42
F1 Score: 0.50
ROC-AUC Score: 0.52
Confusion Matrix:
[[15  9]
 [21 15]]
Image: pred0.jpg, Prediction: no, Probability: [1.0000000e+00 3.2214512e-08]
Image: pred1.jpg, Prediction: no, Probability: [9.9991548e-01 8.4484614e-05]
Image: pred10.jpg, Prediction: yes, Probability: [0.08803807 0.9119619 ]
Image: pred11.jpg, Prediction: yes, Probability: [7.558169e-04 9.992442e-01]
Image: pred12.jpg, Prediction: no, Probability: [0.9550909  0.04490911]
Image: pred13.jpg, Prediction: yes, Probability: [5.2699546e-04 9.9947304e-01]
Image: pred14.jpg, Prediction: yes, Probability: [3.1746038e-07 9.9999964e-01]
Image: pred15.jpg, Prediction: no, Probability: [1.0000000e+00 1.8586208e-08]
Image: pred16.jpg, Prediction: yes, Probability: [0.00188888 0.9981111 ]
Image: pred17.jpg, Prediction: no, Probability: [0.99764735 0.002